In [2]:
import sagemaker
import boto3

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::357224784104:role/PersonalizeDemo-SageMakerIamRole-14RN7NH4VP1M5
sagemaker bucket: sagemaker-us-east-1-357224784104
sagemaker session region: us-east-1


In [3]:
!pip install sagemaker --upgrade

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


### Clone Git

In [4]:
#!git clone https://github.com/tloen/alpaca-lora.git
%cd alpaca-lora
!pip install -r requirements.txt

/home/ec2-user/SageMaker/lora-test/alpaca-lora
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-c1fcz0xm
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-c1fcz0xm
  Resolved https://github.com/huggingface/peft.git to commit 2822398fbe896f25d4dac5e468624dc5fd65a51b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-tgmd0zr_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-tgmd0zr_
  Resolved https://github.com/huggingface/transformers.git to commit d04ec99bec8a0b432fc03ed60cea9a1a20ebaf3c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparin

In [5]:
#!pip uninstall bitsandbytes 
#!pip install bitsandbytes
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


### Local Training

In [ ]:
!python finetune.py \
    --base_model 'decapoda-research/llama-7b-hf' \
    --data_path 'yahma/alpaca-cleaned' \
    --output_dir '../lora-alpaca-ckpt' 


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117_nocublaslt.so
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/ec2-user/anaconda3/envs/pytorch_p39/lib/libcudart.so'), PosixPath('/home/ec2-user/anaconda3/envs/pytorch_p39/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only on

### Fine Tune on SageMaker

In [ ]:
base_model_sm = 'decapoda-research/llama-7b-hf'
#data_path_sm = '/opt/ml/input/data/training'
output_dir_sm = '/opt/ml/model'

In [ ]:
hyperparams = {'base_model': base_model_sm,
                 #'data_path': data_path_sm,
                 'output_dir': output_dir_sm
                 }

In [ ]:
import time
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role

role = get_execution_role()
job_name = f'lora-alpaca-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'

instance_count = 1

# create the Estimator
pytorch_estimator = PyTorch(
    entry_point          = 'finetune.py',          # user endpoint script
    source_dir           = '.',               # directory which includes all the files needed for training
    instance_type        = 'ml.p3.16xlarge', # instances type used for the training job
    instance_count       = instance_count,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    pytorch_version      = '1.13',            # the pytorch_version version used in the training job
    py_version           = 'py39',            # the python version used in the training job
    hyperparameters      = hyperparams,
)

pytorch_estimator.fit()